In [1]:
from typing import Optional, Dict, Iterable, List, Tuple

import hypothesis.strategies
import numpy as np
from gl0learn import fit, synthetic
import pytest
from gl0learn.metrics import nonzeros
from hypothesis import given, settings, HealthCheck, assume
from hypothesis.strategies import composite, just, booleans, floats, integers

In [2]:
@composite
def random_penalty(draw,
                   l0: hypothesis.strategies.SearchStrategy[bool],
                   l1: hypothesis.strategies.SearchStrategy[bool],
                   l2: hypothesis.strategies.SearchStrategy[bool],
                   ) -> List[str]:
    penalties = []

    if draw(l0):
        penalties.append('l0')

    if draw(l1):
        penalties.append('l1')

    if draw(l2):
        penalties.append('l2')

    return penalties


@composite
def random_penalty_values(draw,
                          values_strategies: Dict[str, hypothesis.strategies.SearchStrategy[float]],
                          penalty_strategies: hypothesis.strategies.SearchStrategy[Iterable[str]]
                          ) -> hypothesis.strategies.SearchStrategy[Dict[str, float]]:

    penalties = draw(penalty_strategies)
    values = {}
    for penalty in penalties:
        values[penalty] = draw(values_strategies[penalty])

    return values

In [3]:
penalty_strategty = random_penalty(l0=just(True), l1=booleans(), l2=booleans())

random_penalty_values

<function __main__.accept.<locals>.random_penalty_values(values_strategies: Dict[str, hypothesis.strategies.SearchStrategy[float]], penalty_strategies: hypothesis.strategies.SearchStrategy[typing.Iterable[str]]) -> hypothesis.strategies.SearchStrategy[typing.Dict[str, float]]>

In [4]:
penalty_strategty.example()

['l0']

In [5]:
s = random_penalty_values(penalty_strategies=penalty_strategty,
                          values_strategies={"l0": floats(0.01, 10), "l1": floats(0.01, 10), "l2": floats(0.01, 10)})

In [6]:
s.example()

{'l0': 1.0531029574789406, 'l1': 0.01, 'l2': 0.01}

In [3]:
def top_n_triu_indicies(x, n):
    p = x.shape[1]
    x = np.copy(x)
    x[np.tril_indices(p, k=0)] = 0
    value = np.sort(x.flatten())[::-1][n - 1]

    return np.where(x >= value)

def make_bisect_func(desired_nnz: int, Y: np.ndarray, verbose: bool = True, seed: int = 0, **kwargs):
    def inner_bisect(l0):
        np.random.seed(seed)
        fit_gl0learn = fit(Y, l0=l0, **kwargs)
        theta_hat = fit_gl0learn.theta
        np.fill_diagonal(theta_hat, 0)

        nnz = np.count_nonzero(theta_hat) // 2
        cost = desired_nnz - nnz
        if verbose:
            print(f"gl0Learn found solution with {nnz} non-zeros with parameters:")
            print(f"\t l0 = {l0})")
            print(f"\t cost = {cost}")
        return cost

    return inner_bisect

def _sample_data(n: int = 1000, seed: int = 0) -> Tuple[np.ndarray, np.ndarray]:
    """


    Example Data!


    >>>from tabulate import tabulate
    ...import numpy as np
    ...coords = np.array([str(t).replace('(','').replace(')','') for t in zip(*np.nonzero(np.ones([5,5])))]).reshape(5,5)
    ...table = tabulate(coords, tablefmt="fancy_grid")
    ...print(table)
    ╒══════╤══════╤══════╤══════╤══════╕
    │ 0, 0 │ 0, 1 │ 0, 2 │ 0, 3 │ 0, 4 │
    ├──────┼──────┼──────┼──────┼──────┤
    │ 1, 0 │ 1, 1 │ 1, 2 │ 1, 3 │ 1, 4 │
    ├──────┼──────┼──────┼──────┼──────┤
    │ 2, 0 │ 2, 1 │ 2, 2 │ 2, 3 │ 2, 4 │
    ├──────┼──────┼──────┼──────┼──────┤
    │ 3, 0 │ 3, 1 │ 3, 2 │ 3, 3 │ 3, 4 │
    ├──────┼──────┼──────┼──────┼──────┤
    │ 4, 0 │ 4, 1 │ 4, 2 │ 4, 3 │ 4, 4 │
    ╘══════╧══════╧══════╧══════╧══════╛

    Suppose:
        Coordinates (0,1) and (1,2) are the initial support
        Coordinates (0,2) and (1,3) are also in the active set
        Coordinates (0,3) and (1,4) are also in the super active set

    Supplying `theta_truth` as a upper triangular diagonally dominate matrix, we can set which of `theta_hat` should be learned first.

    This allows us to check if fit is behaving as expected!
    """
    N = 5
    mu = np.zeros(N)

    theta_truth_tril = (1 / 8) * np.asarray([[8, 0, 0, 0, 1],
                                             [0, 8, 4, 2, 3],
                                             [0, 0, 8, 6, 5],
                                             [0, 0, 0, 8, 7],
                                             [0, 0, 0, 0, 8]])

    theta_truth = (theta_truth_tril + theta_truth_tril.T) / 2

    rng = np.random.default_rng(seed)
    x = rng.multivariate_normal(mu, cov=np.linalg.inv(theta_truth), size=n)

    return theta_truth, x

def overlap_covariance_matrix(n: int, max_overlaps: int = 1, seed: int = 0, max_iters: int = 1000):
    rng = np.random.RandomState(seed=seed)
    
    row_overlaps = {i: 0 for i in range(n-1)}
    col_overlaps = {i: 0 for i in range(1, n)}
    
    cov = np.eye(n)
    
    for _ in range(max_iters):
        rows = list(row_overlaps.keys())
        
        row_openings = {}
        for row in rows:
            row_openings[row]= sum(1 for k in col_overlaps if k > row)
            
        num_openings = sum(row_openings.values())
        
        if not num_openings:
            break
        

        row_probability = [r/num_openings for r in row_openings.values()]
        row = rng.choice(rows, p=row_probability)
        try:
            col = rng.choice(list(c for c in col_overlaps.keys() if c > row))
        except ValueError:
            continue
        cov[row, col] += 1
        
        row_overlaps[row] += 1
        col_overlaps[col] += 1
        
        row_overlaps = {r: o for (r, o) in row_overlaps.items() if o < max_overlaps}
        col_overlaps = {c: o for (c, o) in col_overlaps.items() if o < max_overlaps}
        
        
    return cov
    
    
def _sample_data2(n: int = 1000, seed: int = 0) -> Tuple[np.ndarray, np.ndarray]:
    p = 5
    mu = np.zeros(p)
    theta_truth_tril = overlap_covariance_matrix(p, 1)

    theta_truth = (theta_truth_tril + theta_truth_tril.T) / 2

    rng = np.random.default_rng(seed)
    x = rng.multivariate_normal(mu, cov=np.linalg.inv(theta_truth), size=n)
    
    return theta_truth, x



In [67]:
def test_cd_example_2(sample_data, n, algorithm, lXs, seed=0):
    theta_truth, x = sample_data

    _, _, _, _, Y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

    fit_kwargs = dict(**lXs, scale_x=False, max_active_set_size=10, initial_active_set=0.,
                      super_active_set=0., algorithm=algorithm)

    f = make_bisect_func(n, Y, seed=seed, **fit_kwargs)

    from scipy.optimize import bisect
    try:
        opt_l0 = bisect(f, a=0, b=10)
    except ValueError:
        assume(False)
    np.random.seed(seed)
    results = fit(Y, l0=opt_l0, **fit_kwargs)

    theta = results.theta
    
    print(f"===================================> NNZS = {nonzeros(theta[np.tril_indices(5, k=-1)]).sum()}")
    assume(nonzeros(theta[np.tril_indices(5, k=-1)]).sum() == n)

    cd_indices = top_n_triu_indicies(results.theta, n)
    indices = top_n_triu_indicies(theta_truth, n)

    if any(theta_truth[cd_indices] == 0):
        # CD algorithm has selected zero items. This can be fine if we ask for more non-zeros than are in theta_truth!
        # Check if indicies is contained in cd_indices
        indices_set = set(zip(*indices))
        cd_indices_set = set(zip(*indices))
        assert cd_indices_set.issuperset(indices_set)
        should_be_zero_indices_set = cd_indices_set - indices_set

        for (i, j) in should_be_zero_indices_set:
            assert theta_truth[i, j] == 0

    else:
        np.testing.assert_array_equal(cd_indices, indices)

In [85]:
lXs={}
test_cd_example_2(_sample_data2(), n = 1, algorithm = 'CD', lXs=lXs)

gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fit 1
fit loop0
current_iter: 1 cur_objective = 4.95906
fit loop1
current_iter: 2 cur_objective = 4.90657
fit loop2
current_iter: 3 cur_objective = 4.89955
fit loop3
current_iter: 4 cur_objective = 4.89858
fit loop4
current_iter: 5 cur_objective = 4.89844
fit loop5
current_iter: 6 cur_objective = 4.89841
fit loop6
current_iter: 7 cur_objective = 4.8984
fit loop7
current_iter: 8 cur_objective = 4.8984
gl0Learn found solution with 10 non-zeros with parameters:
	 l0 = 0.0)
	 cost = -9
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fit 1
fit loop0
current_iter: 1 cur_objective = 7.03075
fit loop1
current_iter: 2 cur_objective = 7.03075
gl0Learn found solution with 0 non-zeros with parameters:
	 l0 = 10.0)
	 cost = 1
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fit 1
fit loop0
current_iter: 1 cur_objective = 7.03075
fit loop1
current_iter: 2 cur_objective = 7.03075
gl0Learn found solution with 0 non-zeros with parameters:
	 l0 = 5.0)
	 cost = 1
gL0LearnFit 

AssertionError: 
Arrays are not equal

Mismatched elements: 2 / 2 (100%)
Max absolute difference: 2
Max relative difference: 2.
 x: array([[3],
       [4]])
 y: array([[1],
       [3]])

In [136]:
n = 1
opt_l0 = 0
algorithm = "CDPSI"
seed = 0

theta_truth, x = _sample_data2()
_, _, _, _, Y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

fit_kwargs = dict(**lXs, scale_x=False, max_active_set_size=1, initial_active_set=np.inf,
                      super_active_set=0., algorithm=algorithm)

np.random.seed(seed)
results = fit(Y, l0=opt_l0, **fit_kwargs)

theta = results.theta

print(f"===================================> NNZS = {nonzeros(theta[np.tril_indices(5, k=-1)]).sum()}")
#assume(nonzeros(theta[np.tril_indices(5, k=-1)]).sum() == n)

cd_indices = top_n_triu_indicies(results.theta, n)
indices = top_n_triu_indicies(theta_truth, n)

gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fitpsi called 
fit 1
fit loop0
current_iter: 1 cur_objective = 7.03075
fit loop1
current_iter: 2 cur_objective = 7.03075
this->params.max_active_set_size = 1
this->active_set.size() = 0
n_to_keep = 1
fit loop2
current_iter: 3 cur_objective = 6.50749
fit loop3
current_iter: 4 cur_objective = 6.48649
fit loop4
current_iter: 5 cur_objective = 6.48387
fit loop5
current_iter: 6 cur_objective = 6.4835
fit loop6
current_iter: 7 cur_objective = 6.48345
fit loop7
current_iter: 8 cur_objective = 6.48344
fit loop8
current_iter: 9 cur_objective = 6.48344
this->params.max_active_set_size = 1
this->active_set.size() = 1
n_to_keep = 0
Pre psi cost: 6.48344 
PSI iter: 0 
PSI iter: 0 Swapping row: 0
psi_row_fit row =  0 
selected super_active_set start =  {0, 1} 
selected super_active_set end =  {1, 2} 
zero_indices =          1
        3
        4
 
non_zero_indices =          2
 
Non Zero Index Loop: (0, 2) 
No swap for (0, 2) 
PSI iter: 0 Swapping row: 1
psi

In [137]:
results.theta

array([[0.97852669, 0.        , 0.48506987, 0.        , 0.        ],
       [0.        , 0.6957046 , 0.        , 0.        , 0.        ],
       [0.48506987, 0.        , 1.00473363, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.50981079, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.65042778]])

In [123]:
theta_truth, x = _sample_data2()
theta_truth

array([[1.     , 0.     , 0.49005, 0.     , 0.     ],
       [0.     , 1.     , 0.     , 0.5    , 0.     ],
       [0.49005, 0.     , 1.     , 0.     , 0.     ],
       [0.     , 0.5    , 0.     , 1.     , 0.495  ],
       [0.     , 0.     , 0.     , 0.495  , 1.     ]])

In [144]:
def overlap_covariance_matrix(n: int, max_overlaps: int = 1, seed: int = 0, max_iters: int = 1000, decay=.99):
    rng = np.random.RandomState(seed=seed)
    
    overlaps = {i: 0 for i in range(n)}    
    cov = np.eye(n)
    
    v = 1
    
    while len(overlaps) >= 2:
        rows = list(overlaps.keys())

        row, col = rng.choice(rows, size=2, replace=False)
        
        overlaps[row] += 1
        overlaps[col] += 1
        
        cov[row, col] += v
        v *= decay
                
        overlaps = {r: o for (r, o) in overlaps.items() if o < max_overlaps}    
        
    return cov
    
    

In [147]:
p = 5
mu = np.zeros(p)
theta_truth_tril = overlap_covariance_matrix(p, 2, decay=.8)

theta_truth = (theta_truth_tril + theta_truth_tril.T) / 2

rng = np.random.default_rng(seed)
x = rng.multivariate_normal(mu, cov=np.linalg.inv(theta_truth), size=n)


In [148]:
theta_truth

array([[1.    , 0.    , 0.9   , 0.    , 0.    ],
       [0.    , 1.    , 0.    , 0.2048, 0.32  ],
       [0.9   , 0.    , 1.    , 0.    , 0.    ],
       [0.    , 0.2048, 0.    , 1.    , 0.256 ],
       [0.    , 0.32  , 0.    , 0.256 , 1.    ]])

In [207]:

@composite
def overlap_covariance_matrix(draw,
                              n: hypothesis.strategies.SearchStrategy[int] = integers(3, 10),
                              seed: hypothesis.strategies.SearchStrategy[int] = integers(0, 2**32 - 1),
                              max_overlaps: int = 1,
                              decay=.99):
    n = draw(n)
    seed = draw(seed)

    overlaps = {i: 0 for i in range(n)}
    cov = np.eye(n)

    v = 1

    rng = np.random.RandomState(seed=seed)
    while len(overlaps) >= 2:
        rows = list(overlaps.keys())

        row, col = rng.choice(rows, size=2, replace=False)

        overlaps[row] += 1
        overlaps[col] += 1

        cov[row, col] += v
        v *= decay

        overlaps = {r: o for (r, o) in overlaps.items() if o < max_overlaps}
        
    cov = (cov + cov.T)/2

    return cov

In [229]:
g = overlap_covariance_matrix(max_overlaps=1, decay=.8)

In [230]:
theta_truth = g.example()
theta_truth

array([[1. , 0. , 0.5, 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. ],
       [0.5, 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0.4],
       [0. , 0. , 0. , 0.4, 1. ]])

In [231]:
def sample_from_cov(cov: np.ndarray, n: int = 1000, seed: int = 9) -> np.ndarray:
    p, p2 = cov.shape
    assert p == p2

    mu = np.zeros(p)
    rng = np.random.default_rng(seed)
    x = rng.multivariate_normal(mu, cov=np.linalg.inv(cov), size=n)

    return x

In [232]:
x = sample_from_cov(theta_truth)
_, _, _, _, Y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)
results = fit(x, l0=0, scale_x=True, max_active_set_size=1, initial_active_set=np.inf, super_active_set=0.)

theta_truth_copy = np.copy(theta_truth)
np.fill_diagonal(theta_truth_copy, 0)
i, j = np.unravel_index(np.argmax(theta_truth_copy), theta_truth.shape)

gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fit 1
fit loop0
current_iter: 1 cur_objective = 6.11474
fit loop1
current_iter: 2 cur_objective = 6.11474
this->params.max_active_set_size = 1
this->active_set.size() = 0
n_to_keep = 1
fit loop2
current_iter: 3 cur_objective = 5.53552
fit loop3
current_iter: 4 cur_objective = 5.50832
fit loop4
current_iter: 5 cur_objective = 5.50442
fit loop5
current_iter: 6 cur_objective = 5.50379
fit loop6
current_iter: 7 cur_objective = 5.50368
fit loop7
current_iter: 8 cur_objective = 5.50366
fit loop8
current_iter: 9 cur_objective = 5.50366
this->params.max_active_set_size = 1
this->active_set.size() = 1
n_to_keep = 0


In [233]:
results.theta[i, j] > np.mean(theta_truth_copy)

True

In [234]:
results.theta

array([[0.96096045, 0.        , 0.50044781, 0.        , 0.        ],
       [0.        , 0.96929411, 0.        , 0.        , 0.        ],
       [0.50044781, 0.        , 0.99139412, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.79390773, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.82343294]])

In [235]:
i, j

(0, 2)

In [236]:
theta_truth

array([[1. , 0. , 0.5, 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. ],
       [0.5, 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0.4],
       [0. , 0. , 0. , 0.4, 1. ]])

In [239]:
p = 10
module = hypothesis.strategies._internal.core.RandomSeeder(0)
nnz = 4
algorithm = 'CD'
lXs = {}

In [252]:
theta_truth

array([[1.    , 0.    , 0.    , 0.2048, 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.    , 1.    , 0.    , 0.    , 0.4   , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.5   , 0.    ],
       [0.2048, 0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.    , 0.4   , 0.    , 0.    , 1.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 1.    , 0.    , 0.    ,
        0.    , 0.32  ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 1.    , 0.256 ,
        0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.256 , 1.    ,
        0.    , 0.    ],
       [0.    , 0.    , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    ,
        1.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.32  , 0.    , 0.    ,
        0.    , 1.    ]])

In [241]:
def overlap_covariance_matrix(n: int,
                              seed: int = 0,
                              max_overlaps: int = 1,
                              decay=.99):

    overlaps = {i: 0 for i in range(n)}
    cov = np.eye(n)

    v = 1

    rng = np.random.RandomState(seed=seed)
    while len(overlaps) >= 2:
        rows = list(overlaps.keys())

        row, col = rng.choice(rows, size=2, replace=False)

        overlaps[row] += 1
        overlaps[col] += 1

        cov[row, col] += v
        v *= decay

        overlaps = {r: o for (r, o) in overlaps.items() if o < max_overlaps}

    cov = (cov + cov.T)/2

    return cov

In [287]:
1 - np.exp(5 - 6)

0.6321205588285577

In [277]:
overlap_covariance_matrix(5, max_overlaps=1, decay=.99)

array([[1.   , 0.   , 0.5  , 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   , 0.495],
       [0.5  , 0.   , 1.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 1.   , 0.   ],
       [0.   , 0.495, 0.   , 0.   , 1.   ]])

In [278]:
np.linalg.eigvalsh(overlap_covariance_matrix(10, max_overlaps=1, decay=.99))

array([0.5      , 0.505    , 0.50995  , 0.5148505, 0.519702 , 1.480298 ,
       1.4851495, 1.49005  , 1.495    , 1.5      ])

In [242]:
theta_truth = overlap_covariance_matrix(n=p, decay=.8)
x = sample_from_cov(theta_truth)

_, _, _, _, Y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

fit_kwargs = dict(**lXs, scale_x=False, max_active_set_size=p*(p-1)//2, initial_active_set=0.,
                  super_active_set=0., algorithm=algorithm)

f = make_bisect_func(nnz, Y, **fit_kwargs)

from scipy.optimize import bisect
try:
    opt_l0 = bisect(f, a=0, b=10)
except ValueError:
    assume(False)

np.random.seed(0)
results = fit(Y, l0=opt_l0, **fit_kwargs)

theta = results.theta

gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fit 1
fit loop0
current_iter: 1 cur_objective = 9.97932
fit loop1
current_iter: 2 cur_objective = 9.97091
fit loop2
current_iter: 3 cur_objective = 9.97034
fit loop3
current_iter: 4 cur_objective = 9.97028
fit loop4
current_iter: 5 cur_objective = 9.97028
gl0Learn found solution with 45 non-zeros with parameters:
	 l0 = 0.0)
	 cost = -41
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fit 1
fit loop0
current_iter: 1 cur_objective = 11.3992
fit loop1
current_iter: 2 cur_objective = 11.3992
gl0Learn found solution with 0 non-zeros with parameters:
	 l0 = 10.0)
	 cost = 4
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fit 1
fit loop0
current_iter: 1 cur_objective = 11.3992
fit loop1
current_iter: 2 cur_objective = 11.3992
gl0Learn found solution with 0 non-zeros with parameters:
	 l0 = 5.0)
	 cost = 4
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fit 1
fit loop0
current_iter: 1 cur_objective = 11.3992
fit loop1
current_iter: 2 cur_objective = 11.3992
gl0Learn fo

In [246]:
np.round(theta, decimals=1)

array([[ 1. ,  0. ,  0. ,  0.2, -0. , -0. , -0. ,  0. , -0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ,  0.4, -0. , -0. , -0. , -0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ,  0. , -0. ,  0. ,  0. ,  0.5,  0. ],
       [ 0.2,  0. ,  0. ,  0.9, -0. ,  0. , -0. ,  0. , -0. ,  0. ],
       [-0. ,  0.4,  0. , -0. ,  1. ,  0. , -0. ,  0. , -0. , -0. ],
       [-0. , -0. , -0. ,  0. ,  0. ,  1. , -0. ,  0. ,  0. ,  0.3],
       [-0. , -0. ,  0. , -0. , -0. , -0. ,  0.9,  0. , -0. , -0. ],
       [ 0. , -0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. , -0. ],
       [-0. , -0. ,  0.5, -0. , -0. ,  0. , -0. ,  0. ,  1.1,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -0. ,  0.3, -0. , -0. ,  0. ,  1. ]])

In [249]:
np.round(theta_truth, decimals=1)

array([[1. , 0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0.4, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. ],
       [0.2, 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.4, 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0.3],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0.3, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.3, 1. , 0. , 0. ],
       [0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 1. ]])

In [7]:
p = 5
seed = 1

theta_truth = overlap_covariance_matrix(n=p, seed=seed, decay=.8)
x = sample_from_cov(n=30*p**2, cov=theta_truth)

_, _, _, _, Y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

possible_active_set = np.where(np.abs(np.triu(theta_truth, k=1)) > 0)

full_super_active_set = np.asarray(possible_active_set).T
idx = np.random.randint(full_super_active_set.shape[0], size=1)
initial_super_active_set = full_super_active_set[idx, :]

results = fit(Y, **lXs, initial_active_set=np.inf, super_active_set=initial_super_active_set,
              max_active_set_size=p**2)

cd_indices = top_n_triu_indicies(results.theta, 1)

TypeError: overlap_covariance_matrix() got an unexpected keyword argument 'decay'